In [1]:
# default_exp core

# Few-shot Learning with GPT-J

> API details.

In [41]:
# export
import os
import pandas as pd

In [42]:
#hide
from nbdev.showdoc import *
import toml
s = toml.load("../.streamlit/secrets.toml", _dict=dict)

## 🤑🤑 Premium Zingers 🤑🤑


Premium zingers are cherry picked, crowd sourced, from stage-one of the web app.

In [51]:
pd.set_option('display.max_colwidth', 0)

In [49]:
pz = pd.read_csv('../media/premium_zingers.csv', encoding="utf_8_sig")
pz.style.set_properties(**{'text-align': 'left'})

,time_utc,insult,comeback,mood
0,13/02/2022 09:54,A real Mighty Pirateâ„¢ doesn't hide behind 1s and 0s!,You can't even count to 10!,ðŸ’–
1,13/02/2022 00:33,"Which is mightier, my quill or my cutlass?",My sword is sharper... and it's a lot longer!,ðŸ’…
2,12/02/2022 22:47,I've spoken with apes that have created better Monkey Island web apps.,"Well, I'm glad to see you're not afraid of change.",â˜ ï¸
3,12/02/2022 19:40,You have a face like a baboon's ass.,"If you think that's a compliment, I'm not surprised.",â˜ ï¸
4,13/02/2022 00:16,I.. did not.. have sexual relations.. with that woman,"Well, I did. And I enjoyed it very much.",ðŸ˜²
5,12/02/2022 22:48,When you're famous they let you do whatever you want. Grab 'em by the pussy!,You'd grab the pussy of your own mother if you could get away with it!,ðŸ˜²
6,13/02/2022 02:08,I think you are really cute! Would you like to go out for a drink sometime?,With you? I'd rather have a root canal!,ðŸ˜­
7,13/02/2022 08:44,Insulting you is a waste of breath,"You can't insult me, I don't breathe.",ðŸ”¥ðŸ”¥
8,13/02/2022 11:15,No one has ever beaten me in a fight!,That's because you're too fat to throw a punch!,ðŸ”¥ðŸ”¥
9,13/02/2022 11:17,I've killed more people than some countries have inhabitants!,Have you ever considered a career in politics?,ðŸ”¥ðŸ”¥


In [50]:
df = pz
df.columns = ['time_utc', 'Insult', 'Comeback', 'mood']
df.head(3)

,time_utc,Insult,Comeback,mood
0,13/02/2022 09:54,A real Mighty Pirateâ„¢ doesn't hide behind 1s and 0s!,You can't even count to 10!,ðŸ’–
1,13/02/2022 00:33,"Which is mightier, my quill or my cutlass?",My sword is sharper... and it's a lot longer!,ðŸ’…
2,12/02/2022 22:47,I've spoken with apes that have created better Monkey Island web apps.,"Well, I'm glad to see you're not afraid of change.",â˜ ï¸


Using `GPT_J` model API from [Nlpcloud](https://nlpcloud.io/home/token)

In [15]:
import nlpcloud
client = nlpcloud.Client("gpt-j", s['nlpcloud_token'], gpu=True)

# example API call
generation = client.generation("""Insult: You fight like a dairy Farmer!
            Comeback: How appropriate. You fight like a cow!
            ###
            Insult: This is the END for you, you gutter crawling cur!
            Comeback: And I've got a little TIP for you, get the POINT?
            ###
            Insult: I've spoken with apes more polite than you!
            Comeback: I'm glad to hear you attended your family reunion!
            ###
            Insult: Soon you'll be wearing my sword like a shish kebab!
            Comeback: First you'd better stop waving it like a feather duster.
            ###
            Insult: I once owned a dog that was smarter than you.
            Comeback: """,
    max_length=100,
    length_no_input=True,
    end_sequence="\n###",
    remove_input=True)

print('\n🔥🔥🔥 ', generation["generated_text"])


🔥🔥🔥   I once owned a cat that was cleaner than you!



##  ⚔️ Generate Swordfight

In [21]:
# export

def create_input_string(insult, df, training_examples=5, intro=False):
    """
    Loops through insult database and creates few-shot learning string
        Skips over input insult/comeback in database
    """
    if intro: s = "The following is an insult battle between two pirates. \n###\n"
    else: s = ""
        
    for i in range(training_examples):
        if df.Insult[i] == insult: # add example from end of pack
            i = training_examples + 1 
        s += f"""Insult: {df.Insult[i]}\nComeback: {df.Comeback[i]}\n###\n"""
    
    return f"{s}Insult: {insult}\nComeback:"


In [27]:
t = create_input_string("People fall at my feet when they see me coming!", df, training_examples=6)
print(t)

Insult: A real Mighty Pirateâ„¢ doesn't hide behind 1s and 0s!
Comeback:  You can't even count to 10!
###
Insult: Which is mightier, my quill or my cutlass?
Comeback:  My sword is sharper... and it's a lot longer!
###
Insult: I've spoken with apes that have created better Monkey Island web apps.
Comeback:  Well, I'm glad to see you're not afraid of change.
###
Insult: You have a face like a baboon's ass.
Comeback:  If you think that's a compliment, I'm not surprised.
###
Insult: I.. did not.. have sexual relations.. with that woman
Comeback:  Well, I did. And I enjoyed it very much.
###
Insult: When you're famous they let you do whatever you want. Grab 'em by the pussy!
Comeback:  You'd grab the pussy of your own mother if you could get away with it!
###
Insult: People fall at my feet when they see me coming!
Comeback:


In [28]:
# export

def generate_comeback(insult, client, df, outputs, training_examples=3):
    """
    Creates input string and sends to nlpcloud for few-shot learning
    """
    print(f'Insult: ☠️ {insult} ☠️\n')
    t = create_input_string(insult, df, training_examples)
    zingers = []
    for i in range(outputs):
        generation = client.generation(t,
            max_length=100,
            length_no_input=True,
            end_sequence="\n###",
            remove_input=True)
        comeback = generation["generated_text"].strip('\n')
        print(f"Comeback: ({i}) {comeback} 🔥🔥🔥\n")
        zingers.append(comeback)
    return zingers


### Zinger Generation ⚓

In [29]:
generate_comeback("People fall at my feet when they see me coming!", client=client, df=df, outputs=2, training_examples=6)

Insult: ☠️ People fall at my feet when they see me coming! ☠️

Comeback: (0)   Then I'll kill ya! 🔥🔥🔥

Comeback: (1)   You look like everyone's best friend when you get angry. 🔥🔥🔥



["  Then I'll kill ya!",
 "  You look like everyone's best friend when you get angry."]

In [30]:
generate_comeback('Every word you say to me is stupid.', client=client, df=df, outputs=3, training_examples=6)

Insult: ☠️ Every word you say to me is stupid. ☠️

Comeback: (0)   I'm really glad you have different standards for the rest of the world too! 🔥🔥🔥

Comeback: (1)   I'm sorry, I didn't hear what you said... 🔥🔥🔥

Comeback: (2)   You're not the only one that thinks that. 🔥🔥🔥



["  I'm really glad you have different standards for the rest of the world too!",
 "  I'm sorry, I didn't hear what you said...",
 "  You're not the only one that thinks that."]

In [32]:
generate_comeback('Why are you such a stupid dumb idiot?', client=client, df=df, outputs=1, training_examples=9)

Insult: ☠️ Why are you such a stupid dumb idiot? ☠️

Comeback: (0)   I'm an idiot, because idiots are the only people who are more stupid than me! 🔥🔥🔥



["  I'm an idiot, because idiots are the only people who are more stupid than me!"]

In [33]:
generate_comeback("Heaven preserve me! You look like something that's died!", client, df=df, outputs=3, training_examples=15)

Insult: ☠️ Heaven preserve me! You look like something that's died! ☠️

Comeback: (0)   Are you sure this is my heaven, and not some kind of Hell? 🔥🔥🔥

Comeback: (1)   Well, why else would people want you dead? 🔥🔥🔥

Comeback: (2)   You say you want to make your way in life, well, good luck! 🔥🔥🔥



['  Are you sure this is my heaven, and not some kind of Hell?',
 '  Well, why else would people want you dead?',
 '  You say you want to make your way in life, well, good luck!']

In [ ]:
# API Cost PAYG @ nlpcloud.io
gpu_api_calls = 182
tokens_generated = 417
print(f'${gpu_api_calls * 0.001 + 0.0001 * tokens_generated}')

$0.2237
